In [ ]:
多线程 和 多进程 的本质区别不在于 是否多计算核心, 而是在于 逻辑运行之间是否有独立的内存空间：
    多线程的线程之间共享内存空间，线程之间可以共享使用内存变量；多进程的进程有各自独立的内存空间，进程之间的交流通过IPC通信，且copy变量到自己内存空间才能使用

多进程也可以是 单计算核心：比如 multiprocessing 模块不依赖 cpu 核数, 通过 时间片轮转调度 实现多进程。但是这样是无法加速 CPU密集任务的，且引入如下额外开销：
    进程创建销毁、内存复制、进程通信（序列化-IPC管道-反序列化）、调度器切换
进程之间若需要共享内存，则需要引入 进程通信（序列化-IPC管道-反序列化），使得各独立内存间各有数据副本。

多线程也可以是 多计算核心：比如 ThreadPoolExecutor 可以创建 操作系统级线程 OS threads, 可以调度线程到不同CPU核心上，从而加速 CPU密集任务。
线程独占资源：栈(每个线程都有自己的调用栈，存储局部变量、返回地址等)、寄存器状态、线程ID，也就是负责自身运行的必须变量
线程即使被调度到不同 CPU核心上，仍然共享以下资源：
    全局变量(gloabl x)、堆内存(obj = myclass() 创建的对象)、打开的文件描述符、网络连接、代码段(程序指令)

线程共享资源是零成本的(即使跨CPU)：它通过传递内存引用的方式，让工作线程直接访问同一块物理内存 ----> 零拷贝、零序列化、零pickle开销
线程共享资源真正的“成本”在 数据竞争/同步 （race）风险：非原子操作可能引发竞态风险，而加锁/原子操作会降低并发

In [ ]:
Linux 的 多进程启动模式   fork: 子进程会继承 父进程的整个 虚拟内存布局(代码段/堆/栈/全局变量), 但是物理内存页没有复制.
Linux 的 写时复制 机制: Copy on Write, 即
    当子进程继承 父进程 虚拟内存 之时, 如果只是 只读 使用, 那么 子/父 进程其实使用的还是同一块物理内存, 即没有触发内存复制.
    当子进程需要往自己内存写入 某对象(从父进程继承), 此时才会触发 CoW（写时复制 Copy on Write）
综上, Linux 的 fork 多进程启动模式, 能共享某些 父进程的物理内存(只读时), 起到 “零拷贝只读并行” 的作用.

windows 的多进程启动模式 spawn: 子进程拥有全新的虚拟内存, 完全与父进程隔离. 任何需要从父进程继承的对象, 都要经过IPC+副本复制

In [ ]:
Python 的多进程，就是 每个进程有各自独立的Python解释器以运行字节码.

Python 的多线程，有一个额外限制: GIL, Global Interpreter Lock，全局解释锁：它限制了python的多线程：同一时刻只有一个Python解释器线程可运行
它在python初期，是为了保护某些竞态条件 才引入的。

In [ ]:
综上，在 python 环境里，什么时候用 多线程？什么时候用 多进程？二者在多核心CPU系统上皆可得到 CPU密集任务加速。

多线程：不需要 任务/任务的输入输出 可pickle，共享内存对象，避免数据拷贝，缺点是 线程之间可能有 竞态风险。所以
    1. 任务存在不可 pickle 的输入输出，用多线程
    2. 任务必须是线程安全的，最好是 各算各、各输出各，完全避免竞态。并且 任务必须能有效绕开 GIL（比如 Cython 或 C-extend 库）
    这样通过 工作线程共享内存（避免数据拷贝），主线程通过共享收集工作线程的输出，可以把 CPU 利用到最大化



多进程：多个python解释器天然绕开GIL，天然内存隔离没有竞态，缺点是 进程启动开销大，进程IPC通信、数据拷贝开销大。所以
    1. 进程级隔离（防崩溃、沙箱），用 多进程
    2. CPU密集任务 无法绕开或释放 GIL的，用 多进程。比如纯python的cpu密集任务、不释放GIL的库等。
       实际情况下，这种CPU密集任务最好有足够的粗粒度，不能太细（算得多粒度就粗，算得少粒度就细），才能摊平单开一个python解释器的成本

在 fork 下, 如果对(来自父进程的)内存是 只读 使用, CoW机制可以避免很多内存复制，使得 多进程 是很多场景的必选, 比如:
大型 sklearn/xgboost 模型 model 调用其 predict 方法: model.predict，它显然 无法绕开 GIL，所以只能用 多进程, 但如果在 fork 机制下,
由于 predict 调用是只读 model 的, 所以不会触发子进程内存复制, 做到真正的“零开销并行加速”。当然前提是要把 model.predict 封装在 顶层函数：
def predict(data):
    return model.predict(data) # .predict方法无法绕开GIL，只读使用，那么 fork 下不会触发内存复制